In [1]:
"""
ПОЛНЫЙ АНАЛИЗ КРИПТОАДОПЦИИ С ИСПРАВЛЕННЫМ ФОРМАТОМ ДАННЫХ (2010-2025)
Включает все показатели из Analiza-1.xlsx и economy2.R
Версия: 2025-05-23
Изменения этой ревизии (v2):
• Улучшена `optimize_int_columns()` – теперь игнорирует NaN и надёжнее
  переводит «целые» float-столбцы в pandas-тип Int64.
• Все вспомогательные DataFrame перед выгрузкой в Excel также проходят
  оптимизацию, чтобы убрать хвост «.0» в листах статистики и корреляций.
• Обновлены docstring и мелкие комментарии.
"""

import os
import datetime as _dt
import warnings
from typing import Tuple, Dict, Any
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import argparse

warnings.filterwarnings("ignore")

# ─────────────────────────── DISPLAY SETTINGS ────────────────────────────
pd.options.display.float_format = lambda x: f"{x:.0f}" if pd.notna(x) and x % 1 == 0 else f"{x:.2f}"
plt.rcParams["font.family"] = ["DejaVu Sans"]  # поддержка кириллицы
plt.rcParams["axes.unicode_minus"] = False

# ──────────────────────────── HELPER FUNCTIONS ───────────────────────────

def optimize_int_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert float columns whose non-NaN values are all integer-like to **Int64**.

    Это устраняет паразитный хвост «.0» в Excel/CSV/Jupyter, сохранив
    пропуски (nullable integer).
    """
    float_cols = df.select_dtypes(include="float").columns
    for col in float_cols:
        ser = df[col]
        if ((ser.dropna() % 1) == 0).all():
            df[col] = ser.astype("Int64")
    return df

def create_project_structure() -> str:
    base = os.path.dirname(__file__)
    rel_path = "2091/data.txt"
    abs_file_path = os.path.join(script_dir, rel_path)
    for sub in ("grafiki", "otchety", "dannye", "rezultaty"):
        os.makedirs(os.path.join(base, sub), exist_ok=True)
        print(f"✅ Создана папка: {os.path.join(base, sub)}")
    return base

In [ ]:
def create_project_structure() -> str:
    base = os.path.join(os.path.dirname(__file__), "../../data/")
    for sub in ("grafiki", "otchety", "dannye", "rezultaty"):
        os.makedirs(os.path.join(base, sub), exist_ok=True)
        print(f"✅ Создана папка: {os.path.join(base, sub)}")
    return base

In [1]:
from function import func

In [6]:
base = func.create_project_structure()

✅ Создана папка: /Users/kirill/Desktop/analiza/app/src/function/../../data/grafiki
✅ Создана папка: /Users/kirill/Desktop/analiza/app/src/function/../../data/otchety
✅ Создана папка: /Users/kirill/Desktop/analiza/app/src/function/../../data/dannye
✅ Создана папка: /Users/kirill/Desktop/analiza/app/src/function/../../data/rezultaty


In [13]:
import pandas as pd
import os
hdi_data = pd.read_csv(os.path.join(base, "hdi_data.csv"))

In [11]:
hdi_data = {
        'Ukraine': [0.710, 0.720, 0.734, 0.734, 0.743, 0.743, 0.751, 0.751, 0.759, 0.759, 0.773, 0.773, 0.734, 0.734, 0.734, 0.734],
        'Poland': [0.813, 0.813, 0.834, 0.834, 0.855, 0.855, 0.865, 0.865, 0.876, 0.876, 0.880, 0.880, 0.876, 0.876, 0.876, 0.876],
        'Czech': [0.861, 0.861, 0.878, 0.878, 0.888, 0.888, 0.900, 0.900, 0.900, 0.900, 0.889, 0.889, 0.889, 0.889, 0.889, 0.889],
        'Sweden': [0.885, 0.885, 0.907, 0.907, 0.933, 0.933, 0.937, 0.937, 0.945, 0.945, 0.947, 0.947, 0.947, 0.947, 0.947, 0.947],
        'Norway': [0.938, 0.938, 0.944, 0.944, 0.949, 0.949, 0.953, 0.953, 0.957, 0.957, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961],
        'Belarus': [0.786, 0.786, 0.796, 0.796, 0.808, 0.808, 0.817, 0.817, 0.823, 0.823, 0.823, 0.823, 0.808, 0.808, 0.808, 0.808]
    }

In [12]:
for country_code, hdi_values in hdi_data.items():
        print(country_code)
        print(hdi_values)
        print("hello")

Ukraine
[0.71, 0.72, 0.734, 0.734, 0.743, 0.743, 0.751, 0.751, 0.759, 0.759, 0.773, 0.773, 0.734, 0.734, 0.734, 0.734]
hello
Poland
[0.813, 0.813, 0.834, 0.834, 0.855, 0.855, 0.865, 0.865, 0.876, 0.876, 0.88, 0.88, 0.876, 0.876, 0.876, 0.876]
hello
Czech
[0.861, 0.861, 0.878, 0.878, 0.888, 0.888, 0.9, 0.9, 0.9, 0.9, 0.889, 0.889, 0.889, 0.889, 0.889, 0.889]
hello
Sweden
[0.885, 0.885, 0.907, 0.907, 0.933, 0.933, 0.937, 0.937, 0.945, 0.945, 0.947, 0.947, 0.947, 0.947, 0.947, 0.947]
hello
Norway
[0.938, 0.938, 0.944, 0.944, 0.949, 0.949, 0.953, 0.953, 0.957, 0.957, 0.961, 0.961, 0.961, 0.961, 0.961, 0.961]
hello
Belarus
[0.786, 0.786, 0.796, 0.796, 0.808, 0.808, 0.817, 0.817, 0.823, 0.823, 0.823, 0.823, 0.808, 0.808, 0.808, 0.808]
hello


In [10]:
for country_code, hdi_values in hdi_data.items():
        print(country_code)
        print(hdi_values)
        print("hello")
        #countries_data[country_code]['hdi'] = hdi_values

Country
0     Ukraine
1     Ukraine
2     Ukraine
3     Ukraine
4     Ukraine
       ...   
91    Belarus
92    Belarus
93    Belarus
94    Belarus
95    Belarus
Name: Country, Length: 96, dtype: object
hello
Year
0     2000
1     2001
2     2002
3     2003
4     2004
      ... 
91    2011
92    2012
93    2013
94    2014
95    2015
Name: Year, Length: 96, dtype: int64
hello
HDI
0    0.71
1    0.72
2    0.73
3    0.73
4    0.74
     ... 
91   0.82
92   0.81
93   0.81
94   0.81
95   0.81
Name: HDI, Length: 96, dtype: float64
hello
